In [1]:
import sys, os, math
sys.path.append('/mnt/volume/shared/anna/hillary/CMSCB-SCEMILA/SCEMILA/ml_pipeline')
import statistics
import numpy as np
import pandas as pd
import pickle as pkl
import os.path
from tqdm import tqdm
import torch
import torch.nn.functional as F
from dataset_mixed import *  # dataset
from model import *  # actual MIL model
from sklearn import metrics as metrics
import csv

import umap

import label_converter

# import functions instead of having them in here to keep the notebook *much* shorter
sys.path.append('functions')


/tmp/ipykernel_390692/2773219413.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
CLASSES = ['control', 'RUNX1_RUNX1T1', 'NPM1', 'CBFB_MYH11', 'PML_RARA']
SOURCE_FOLDER = "/mnt/volume/shared/test_data"
patients = {}
with open(os.path.join(SOURCE_FOLDER,'metadata.csv'), newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        # print(line)
        patients[line[1]] = [os.path.join(SOURCE_FOLDER,
                                         "data",
                                         line[4],
                                         line[1],
                                         "fnl34_bn_features_layer_7.npy"), line[4]]

In [3]:
def classwise_area_under_pr(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        y_pred_class = predictions[:, class_index]
        y_true_class = ground_truth[:, class_index]

        # Compute precision-recall curve
        precision, recall, _ = metrics.precision_recall_curve(y_true_class, y_pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = metrics.auc(recall, precision)

    return classwise_auc

def classwise_area_under_roc(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        y_pred_class = predictions[:, class_index]
        y_true_class = ground_truth[:, class_index]

        # Compute precision-recall curve
        score = metrics.roc_auc_score(y_true_class, y_pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = score

    return classwise_auc

def classwise_acc(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        y_pred_class = predictions[:, class_index]
        y_true_class = ground_truth[:, class_index]

        # Compute precision-recall curve
        score = metrics.accuracy(y_true_class, y_pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = score

    return classwise_auc

def classwise_f1(predictions, ground_truth, num_classes):
    classwise_auc = np.zeros(num_classes)

    for class_index in range(num_classes):
        # Extract the predictions and ground truth for the current class
        pred_class = (predictions == class_index).astype(int)
        gt_class = (ground_truth == class_index).astype(int)

        # Compute precision-recall curve
        score = metrics.f1_score(gt_class, pred_class)

        # Compute area under the PR curve for the current class
        classwise_auc[class_index] = score

    return classwise_auc

def calculate_metrics(pred,gt, classes):
    pred = np.array(pred).squeeze()
    gt = np.array(gt)
    pred0 = np.argmax(pred, axis=1)
    gt0 = np.argmax(gt, axis=1)
    acc = metrics.accuracy_score(gt0, pred0)
    f1 = metrics.f1_score(gt0, pred0, average="micro")
    matrix = metrics.confusion_matrix(gt0, pred0)
    c_acc = matrix.diagonal()/matrix.sum(axis=1)
    print(metrics.classification_report(gt0, pred0, target_names=classes))
    c_f1 = classwise_f1(pred0,gt0, 5)

    auc_pr = classwise_area_under_pr(pred, gt, 5)
    auc_roc = classwise_area_under_roc(pred, gt, 5)

    # print("Area under the PR curve for each class:", auc_pr)

    return acc, f1, auc_pr, auc_roc, c_acc, c_f1

def correct_order_classes(converter,auc):
    out = np.zeros(len(CLASSES))

    for i in range(len(CLASSES)):
        out[i] = auc[converter[CLASSES[i]]]

    return out

In [4]:
def get_preds(patients, model, class_converter):
    pred = []
    gt = []
    for p in patients.keys():
            path, lbl_name = patients[p]
            lbl = np.zeros(5)
            lbl[class_converter[lbl_name]] = 1

            bag = np.load(path)

            bag = torch.tensor(bag)
            bag = torch.unsqueeze(bag,0)
            p, _, _, _ = model(bag)
            pred.append(F.softmax(p,dim=1).cpu().detach().numpy())
            gt.append(lbl)
    return pred, gt


In [18]:
class_converter = {}
with open("/mnt/volume/shared/class_conversion.csv", newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)
    for line in reader:
        class_converter[line[1]] = int(line[0])
paths = {"/mnt/volume/shared/all_results/exp0/result_folder_1": "exp0",
         "/mnt/volume/shared/new_results/experiment_3_seed1/result_folder": "exp3_seed1",
         "/mnt/volume/shared/new_results/experiment_3_seed20/result_folder": "exp3_seed20",
         "/mnt/volume/shared/new_results/mixed_seed1/10_percent/result_folder": "mixed_seed1_10percent",
         "/mnt/volume/shared/new_results/mixed_seed20/10_percent/result_folder": "mixed_seed20_10percent",
         "/mnt/volume/shared/new_results/mixed_seed1/20_percent/result_folder": "mixed_seed1_20percent",
         "/mnt/volume/shared/new_results/mixed_seed20/20_percent/result_folder": "mixed_seed20_20percent",
         "/mnt/volume/shared/new_results/mixed_seed1/30_percent/result_folder": "mixed_seed1_30percent",
         "/mnt/volume/shared/new_results/mixed_seed20/30_percent/result_folder": "mixed_seed20_30percent",
         "/mnt/volume/shared/new_results/mixed_seed1/50_percent/result_folder": "mixed_seed1_50percent",
         "/mnt/volume/shared/new_results/mixed_seed20/50_percent/result_folder": "mixed_seed20_50percent"
        }
inv_class = {v: k for k, v in class_converter.items()}
classes = []
for i in range(5):
    classes.append(inv_class[i])

In [23]:
df = pd.DataFrame(columns=['path','acc','f1','auc_pr','auc_roc','c_acc','c_f1'])
for path in paths.keys():
    model = torch.load(os.path.join(path,"state_dictmodel.pt"),map_location="cpu")
    pred, gt = get_preds(patients, model, class_converter)
    acc, f1, auc_pr , auc_roc, c_acc, c_f1 = calculate_metrics(pred, gt, classes)
    df.loc[len(df.index)] = [paths[path], acc, f1, ['%.2f' % elem for elem in auc_pr] , ['%.2f' % elem for elem in auc_roc], ['%.2f' % elem for elem in c_acc], ['%.2f' % elem for elem in c_f1]]

               precision    recall  f1-score   support

RUNX1_RUNX1T1       1.00      0.67      0.80         6
      control       1.00      1.00      1.00        12
         NPM1       0.86      0.86      0.86         7
     PML_RARA       1.00      1.00      1.00         5
   CBFB_MYH11       0.67      0.86      0.75         7

     accuracy                           0.89        37
    macro avg       0.90      0.88      0.88        37
 weighted avg       0.91      0.89      0.89        37

               precision    recall  f1-score   support

RUNX1_RUNX1T1       0.50      0.50      0.50         6
      control       1.00      1.00      1.00        12
         NPM1       0.50      0.57      0.53         7
     PML_RARA       1.00      1.00      1.00         5
   CBFB_MYH11       0.83      0.71      0.77         7

     accuracy                           0.78        37
    macro avg       0.77      0.76      0.76        37
 weighted avg       0.79      0.78      0.79        37

    

In [24]:
df

,path,acc,f1,auc_pr,auc_roc,c_acc,c_f1
0,exp0,0.891892,0.891892,"[0.92, 1.00, 0.89, 1.00, 0.82]","[0.98, 1.00, 0.94, 1.00, 0.96]","[0.67, 1.00, 0.86, 1.00, 0.86]","[0.80, 1.00, 0.86, 1.00, 0.75]"
1,exp3_seed1,0.783784,0.783784,"[0.50, 1.00, 0.72, 1.00, 0.93]","[0.91, 1.00, 0.91, 1.00, 0.98]","[0.50, 1.00, 0.57, 1.00, 0.71]","[0.50, 1.00, 0.53, 1.00, 0.77]"
2,exp3_seed20,0.810811,0.810811,"[0.61, 1.00, 0.79, 1.00, 0.91]","[0.88, 1.00, 0.93, 1.00, 0.97]","[0.67, 1.00, 0.57, 0.80, 0.86]","[0.57, 1.00, 0.67, 0.89, 0.80]"
3,mixed_seed1_10percent,0.891892,0.891892,"[0.92, 1.00, 0.90, 1.00, 0.97]","[0.97, 1.00, 0.97, 1.00, 0.99]","[0.83, 1.00, 0.86, 0.80, 0.86]","[0.83, 1.00, 0.86, 0.89, 0.80]"
4,mixed_seed20_10percent,0.891892,0.891892,"[0.96, 1.00, 0.91, 1.00, 0.76]","[0.99, 1.00, 0.98, 1.00, 0.96]","[0.83, 1.00, 0.86, 0.80, 0.86]","[0.83, 1.00, 0.86, 0.89, 0.80]"
5,mixed_seed1_20percent,0.918919,0.918919,"[1.00, 1.00, 0.89, 1.00, 0.84]","[1.00, 1.00, 0.95, 1.00, 0.97]","[1.00, 1.00, 0.86, 0.80, 0.86]","[0.92, 1.00, 0.86, 0.89, 0.86]"
6,mixed_seed20_20percent,0.891892,0.891892,"[0.96, 1.00, 0.87, 1.00, 0.78]","[0.99, 1.00, 0.94, 1.00, 0.97]","[0.83, 1.00, 0.71, 0.80, 1.00]","[0.77, 1.00, 0.83, 0.89, 0.88]"
7,mixed_seed1_30percent,0.864865,0.864865,"[1.00, 1.00, 0.92, 1.00, 0.86]","[1.00, 1.00, 0.98, 1.00, 0.96]","[0.67, 1.00, 0.71, 1.00, 0.86]","[0.80, 1.00, 0.77, 1.00, 0.71]"
8,mixed_seed20_30percent,0.864865,0.864865,"[0.88, 1.00, 0.91, 1.00, 0.86]","[0.96, 1.00, 0.97, 1.00, 0.97]","[0.67, 1.00, 0.86, 0.80, 0.86]","[0.73, 1.00, 0.86, 0.89, 0.75]"
9,mixed_seed1_50percent,0.918919,0.918919,"[0.86, 1.00, 0.84, 1.00, 0.84]","[0.98, 1.00, 0.92, 1.00, 0.97]","[1.00, 1.00, 0.71, 1.00, 0.86]","[0.92, 1.00, 0.77, 1.00, 0.86]"
